---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [36]:
# # Cleaning functions
# remove_parenthesis = lambda name: name.split(' (')[0] if ' (' in name else name
# remove_brackets = lambda name: name.split('[')[0] if '[' in name else name

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    # Reading university_towns.txt indo a DataFrame
    with open('university_towns.txt', 'r') as uni_txt:
        uni_list = uni_txt.readlines()
    # Treating data
    # Checking if it is a state or a town and treating according
    uni_tuples_list = []
    state = ''
    for name in uni_list:
        name = name.rstrip('\n')
        if '[edit]' in name:
            name = name.split('[')[0]
            state = name
        else:
            if ' (' in name:
                name = name.split(' (')[0]
            uni_tuples_list.append((state, name))
    # Creating uni_df
    uni_df = pd.DataFrame(uni_tuples_list, columns=['State', 'RegionName'])

    return uni_df

# get_list_of_university_towns()

In [32]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a
    string value in a format such as 2005q3'''
    # Reading GDP DataFrame from xls file
    GDP = pd.read_excel('gdplev.xls', skiprows=7)
    # Selecting only the columns of interest
    GDP = GDP[['Unnamed: 4', 'Unnamed: 6']]
    # Defining columns and renaming it
    GDP.columns = ['Quarters', 'GDP 2009']
    # Removing data from before 2000
    GDP = GDP[GDP['Quarters'] >= '2000q1']
    # Looping through DataFrame to find the recession start
    recession_start = ''
    for i in range(GDP.shape[0] - 2):
        if (GDP.iloc[i][1] > GDP.iloc[i+1][1] and
            GDP.iloc[i+1][1] > GDP.iloc[i+2][1]):
            recession_start = GDP.iloc[i+1][0]
            break

    return recession_start

# get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    # Reading GDP DataFrame from xls file
    GDP = pd.read_excel('gdplev.xls', skiprows=7)
    # Selecting only the columns of interest
    GDP = GDP[['Unnamed: 4', 'Unnamed: 5']]
    # Defining columns and renaming it
    GDP.columns = ['Quarters', 'GDP 2009']
    # Grabbing the recession start
    recession_start = get_recession_start()
    # Removing data from before 2000
    GDP = GDP[GDP['Quarters'] >= recession_start]
    # Looping through DataFrame to find the recession end
    recession_end = ''
    for i in range(GDP.shape[0] - 2):
        if (GDP.iloc[i][1] < GDP.iloc[i+1][1] and
            GDP.iloc[i+1][1] < GDP.iloc[i+2][1]):
            recession_end = GDP.iloc[i+2][0]
            break

    return recession_end
       
# get_recession_end()

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    # Reading GDP DataFrame from xls file
    GDP = pd.read_excel('gdplev.xls', skiprows=7)
    # Selecting only the columns of interest
    GDP = GDP[['Unnamed: 4', 'Unnamed: 5']]
    # Defining columns and renaming it
    GDP.columns = ['Quarters', 'GDP 2009']
    # Grabbing recession start and end to calculate the bottom
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    # Removing unnecessary data
    recession = GDP[GDP['Quarters'] >= recession_start]
    recession = recession[recession['Quarters'] <= recession_end]
    # Resetting index
    recession.reset_index(drop=True, inplace=True)

    return recession.iloc[recession['GDP 2009'].idxmin][0]

# get_recession_bottom()

In [8]:
def convert(month):
    month_dict = {'01': 'q1',
                  '04': 'q2',
                  '07': 'q3',
                  '10': 'q4'}

    return month_dict[month]

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Reading data pro City_Zhvi_AllHomes.csv file
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    #  Dropping unwanted data and setting its new index
    df['State'] = df['State'].map(states)
    df = (df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
            .drop([col for col in df.columns if col < '2000-01'], axis=1)
            .set_index(['State', 'RegionName']))
    # Generating new quarter columns and dropping month ones
    df_columns = df.columns
    for i in range(0, df.shape[1]-1, 3):
        # Slicing the original DataFrame
        aux_df = df.iloc[:,i:i+3]
        # Grabbing year and month
        year, month = aux_df.columns[0].split('-')
        # Mapping the month to a quarter start
        quarter = convert(month)
        # Creating quarter column
        df['{}{}'.format(year, quarter)] = aux_df.mean(axis=1)

    # Dropping month columns
    df.drop(df_columns, axis=1, inplace=True)

    return df

# convert_housing_data_to_quarters()

In [19]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # Loading data
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    university_towns = get_list_of_university_towns()
    hd_quarters = convert_housing_data_to_quarters()    
    # Price variation
    hd_quarters = hd_quarters[[recession_start, recession_bottom]]
    hd_quarters['PriceVariation'] = hd_quarters.iloc[:,1] - hd_quarters.iloc[:,0]
    hd_quarters.drop(hd_quarters.columns[:2], axis=1, inplace=True)
    # Joining DataFrames to check if it is a University Town
    final_df = pd.merge(hd_quarters, university_towns, left_index=True,
                        right_on=['State', 'RegionName'], how='left',
                        indicator='IsUniversityTown')
    final_df['IsUniversityTown'] = np.where(final_df.IsUniversityTown == 'both', True, False)
    final_df.drop(['State', 'RegionName'], axis=1, inplace=True)
    # Calculating the finals results (different, p, better)
    unitowns = final_df[final_df['IsUniversityTown'] == True].dropna()
    not_unitowns = final_df[final_df['IsUniversityTown'] == False].dropna()
    t, p = ttest_ind(unitowns['PriceVariation'], not_unitowns['PriceVariation'])
    different = True if p < .01 else False
    utm = abs(unitowns['PriceVariation'].mean())
    nutm = abs(not_unitowns['PriceVariation'].mean())
    better = 'university town' if utm < nutm else 'non-university town'
    
    return (different, p, better)

# run_ttest()

(True, 0.00324803036863141, 'university town')